In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('vaccine_preprocessed.csv',sep='|')

In [3]:
df.head(1)

,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
0,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,0.0,"['澎湖5至11歲兒童疫苗接種種率已逾7成', '澎湖縣政府今天公布新增182例確診案例',...","[('澎湖', 5), ('疫苗', 5), ('接種', 4), ('確診', 3), (...","['澎湖', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾...","['澎湖', '冠狀', '病毒', '疾病', '確診數', '澎湖縣', '政府', '...","[NerToken(word='澎湖', ner='GPE', idx=(0, 2)), N...","[('澎湖', 'Nc'), ('COVID-19', 'FW'), ('（', 'PARE...",https://news.google.com/articles/CBMiMmh0dHBzO...


In [5]:
def filter_df_via_tokens(query_keywords, cond, cate):
    
    # proceed filtering
    if (cate == "全部"):
        condition = (df.categories == 'AZ') | (df.categories == 'BNT') | (df.categories == '莫德納') | (df.categories == '高端')# "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = df.categories == cate

    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.tokens_v2.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.tokens_v2.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [8]:
def filter_df_via_content(query_keywords, cond, cate):
    
    # proceed filtering
    if (cate == "全部"):
        condition = (df.categories == 'AZ') | (df.categories == 'BNT') | (df.categories == '莫德納') | (df.categories == '高端')# "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = df.categories == cate

    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.contents.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.contents.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query

In [7]:
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.6: #大於0.6為正向 
            sentiCount['pos'] += 1
        elif senti <= 0.4: #小於0.4為負向
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1 #介於中間為中立
    for polar in sentiCount :
        try:
            sentiPercnt[polar]=int(sentiCount[polar]/numberOfArticle*100)
            # sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle,2) # 0.75
        except:
            sentiPercnt[polar] = 0 # 若分母 numberOfArticle=0會報錯
    return sentiCount, sentiPercnt

In [9]:
query_keywords = ['陳時中']
cond='or'
cate='全部'
df_query = filter_df_via_content(query_keywords, cond, cate)
sentiCount, sentiPercnt = get_article_sentiment(df_query)

In [10]:
sentiCount

{'pos': 3, 'neg': 226, 'obj': 1}

In [11]:
sentiPercnt

{'pos': 1, 'neg': 98, 'obj': 0}

In [12]:
'正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),str(sentiPercnt['neg']))

'正向:1%, 中立:0%, 負向:98%'

In [13]:
[sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

[1, 0, 98]

In [18]:
def prepare_for_response(query_keywords, cond, cate):

    # Proceed filtering
    df_query = filter_df_via_content(query_keywords, cond, cate)
    
    sentiCount, sentiPercnt = get_article_sentiment(df_query)

    response = {
        'sentiCount': sentiCount,
    }
    return response

In [19]:
query_keywords = ['陳時中']
cond='or'
cate='全部'
prepare_for_response(query_keywords, cond, cate)

{'sentiCount': {'pos': 3, 'neg': 226, 'obj': 1}}

## Djnago function

In [ ]:
from django.http import JsonResponse
from django.shortcuts import render
import pandas as pd
from datetime import datetime, timedelta
from django.views.decorators.csrf import csrf_exempt


# (1) Load news data--approach 1 直接指定某個csv檔案
def load_df_data():
    # global variable
    global  df
    df = pd.read_csv('app_top_keyword_association/dataset/vaccine_preprocessed.csv', sep='|')

load_df_data()

def home(request):
    return render(request, 'app_user_keyword_sentiment/home.html')

# GET: csrf_exempt is not necessary
# POST: csrf_exempt should be used
@csrf_exempt
def api_get_userkey_sentiment(request):

    userkey = request.POST['userkey']
    cate = request.POST['cate']
    cond = request.POST['cond']

    query_keywords = userkey.split()
    response = prepare_for_response(query_keywords, cond, cate)
  
    return JsonResponse(response)

def prepare_for_response(query_keywords, cond, cate, weeks):

    # Proceed filtering
    df_query = filter_df_via_content(query_keywords, cond, cate)
    
    sentiCount, sentiPercnt = get_article_sentiment(df_query)

    response = {
        'sentiCount': sentiCount,
    }
    return response

def get_article_sentiment(df_query):
    sentiCount = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
    sentiPercnt = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # determine sentimental polarity
        if float(senti) >= 0.6:
            sentiCount['Positive'] += 1
        elif float(senti) <= 0.4:
            sentiCount['Negative'] += 1
        else:
            sentiCount['Neutral'] += 1
    for polar in sentiCount :
        try:
            sentiPercnt[polar]=int(sentiCount[polar]/numberOfArticle*100)
        except:
            sentiPercnt[polar] = 0 # 若分母 numberOfArticle=0會報錯
    return sentiCount, sentiPercnt


# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_df_via_content(query_keywords, cond, cate):

    # proceed filtering
    if (cate == "全部"):
        condition = (df.categories == 'AZ') | (df.categories == 'BNT') | (df.categories == '莫德納') | (df.categories == '高端')# "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = df.categories == cate

    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.contents.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.contents.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


print("app_userkey_sentiment was loaded!")


